# Kayıp Fonksiyonları

Bu alıştırmada, Kayıp fonksiyonlarının `LinearRegression` modeli üzerindeki etkilerini karşılaştıracaksınız.

👇 Bu zorluk için kullanmak üzere bir CSV dosyası indirelim ve onu bir DataFrame'e dönüştürelim

In [1]:
import pandas as pd

data = pd.read_csv("https://d32aokrjazspmn.cloudfront.net/materials/loss_functions_dataset.csv")
data.sample(5)

,Relative Compactness,Surface Area,Wall Area,Roof Area,Overall Height,Glazing Area,Average Temperature
167,0.76,661.5,416.5,122.5,7.0,0.10,33.870
668,0.62,808.5,367.5,220.5,3.5,0.40,16.685
420,0.66,759.5,318.5,220.5,3.5,0.25,14.250
37,0.66,759.5,318.5,220.5,3.5,0.00,9.665
611,0.69,735.0,294.0,220.5,3.5,0.40,15.920


🎯 Göreviniz, tasarımına göre bir seranın içindeki ortalama sıcaklığı tahmin etmektir. Sıcaklık tahminleriniz, her bir bitki için iklim ihtiyaçlarına göre uygun sera tasarımını seçmenize yardımcı olacaktır.

🌿 Bitkilerin küçük sıcaklık değişimlerini kaldırabildiğini, ancak sıcaklık değişimleri arttıkça katlanarak daha duyarlı hale geldiğini biliyorsunuz.

## 1. Teori

❓ Teorik olarak, bitkileri öldürme riskini sınırlamak için modelinizi hangi Kayıp fonksiyonu üzerinde eğitirsiniz?

<details>
<summary> 🆘 Cevap </summary>
    
Teorik olarak, Ortalama Kare Hata (MSE) Kayıp fonksiyonunu kullanırsınız. Bu, aykırı tahminleri cezalandırır ve modelinizin büyük hatalar yapmasını engeller. Bu, daha küçük sıcaklık değişimleri ve bitkiler için daha düşük risk sağlayacaktır.

</details>

Bu problemde amaç, bitkiler için risk oluşturabilecek büyük sıcaklık sapmalarını mümkün olduğunca azaltmaktır. 
Bitkiler küçük sıcaklık değişimlerini tolere edebilse de, büyük tahmin hataları ciddi verim kayıplarına ve bitkilerin zarar görmesine neden olabilir.

Bu nedenle büyük hataları karesel olarak daha fazla cezalandıran Ortalama Kare Hata (MSE) kayıp fonksiyonu tercih edilmelidir. 
MSE, modelin aşırı hatalı tahminler yapmasını engelleyerek daha güvenli ve istikrarlı sıcaklık tahminleri üretmesini sağlar.

## 2. Uygulama

### 2.1 Ön İşleme

❓ Özellikleri standartlaştırın

In [2]:
from sklearn.preprocessing import StandardScaler

X = data.drop("Average Temperature", axis=1)
y = data["Average Temperature"]

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

### 2.2 Modelleme

Bu bölümde, farklı Kayıp fonksiyonları üzerinde optimize edilmiş modelleri değerlendirerek teoriyi doğrulayacaksınız.

### En Küçük Kareler (MSE) Kaybı

❓ **En Küçük Kareler Kaybı** (MSE) üzerinde **Stokastik Gradyan İnişi** (SGD) ile optimize edilmiş bir Doğrusal Regresyon modelini **10-Katlı Çapraz doğrula**

In [3]:
from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import cross_val_score

sgd_mse = SGDRegressor(
    loss="squared_error",
    max_iter=1000,
    tol=1e-3,
    random_state=42
)

r2_scores = cross_val_score(
    sgd_mse,
    X_scaled,
    y,
    cv=10,
    scoring="r2"
)

r2 = r2_scores.mean()
r2

0.8982272875896993

❓ Hesaplayın:
- Ortalama çapraz doğrulanmış R2 skoru ve bunu `r2` değişkeninde kaydedin
- Tüm katlarınızın °C cinsinden en büyük tek tahmin hatasını hesaplayın ve `max_error_celsius` değişkeninde kaydedin

(İpucu: `max_error` sklearn'de kabul edilen bir puanlama metriğidir)

In [4]:
from sklearn.metrics import max_error
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42
)

sgd_mse.fit(X_train, y_train)
y_pred = sgd_mse.predict(X_test)

max_error_celsius = max_error(y_test, y_pred)
max_error_celsius

9.136012771233176

### Ortalama Mutlak Hata (MAE) Kaybı

Peki modelimizi MAE üzerinde optimize edersek ne olur?

❓ **MAE** Kaybı üzerinde **Stokastik Gradyan İnişi** (SGD) ile optimize edilmiş bir Doğrusal Regresyon modelini **10-Katlı Çapraz doğrula**

<details>
<summary>💡 İpuçları</summary>

- MAE kaybı `SGDRegressor`'da doğrudan belirtilemez. Doğru parametreleri ayarlayarak tasarlanması gerekir

</details>

In [5]:
from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import cross_val_score

sgd_mae = SGDRegressor(
    loss="epsilon_insensitive",
    epsilon=0.1,
    max_iter=1000,
    tol=1e-3,
    random_state=42
)

r2_scores_mae = cross_val_score(
    sgd_mae,
    X_scaled,
    y,
    cv=10,
    scoring="r2"
)

r2_mae = r2_scores_mae.mean()
r2_mae

0.8755826687124214

❓ Hesaplayın:
- Ortalama çapraz doğrulanmış R2 skoru, bunu `r2_mae`'de saklayın
- Tüm katlarınızın en büyük tek tahmin hatasını, bunu `max_error_mae`'de saklayın

In [6]:
from sklearn.metrics import max_error
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42
)

sgd_mae.fit(X_train, y_train)
y_pred_mae = sgd_mae.predict(X_test)

max_error_mae = max_error(y_test, y_pred_mae)
max_error_mae

11.438089792572665

## 3. Sonuç

❓ Değerlendirdiğiniz modellerden hangisi göreviniz için en uygun görünüyor?

<details>
<summary> 🆘Cevap </summary>
    
İki model arasında ortalama çapraz doğrulanmış r2 skorları yaklaşık olarak benzer olmasına rağmen, MAE üzerinde optimize edilen modelin zaman zaman daha büyük hatalar yapma şansı daha fazladır, bu da bitkileri öldürme riskini artırır!
    
</details>

Değerlendirilen iki modelin ortalama çapraz doğrulanmış R² skorları birbirine yakın olsa da,
MAE kaybı ile optimize edilen modelin bazı durumlarda daha büyük tahmin hataları ürettiği
gözlemlenmiştir. Bu durum, sıcaklık tahminlerindeki büyük sapmaların bitkiler için
yüksek risk oluşturması nedeniyle istenmeyen bir sonuçtur.

Bu nedenle büyük hataları daha sert biçimde cezalandıran MSE kaybı ile optimize edilen
model, bitkilerin sıcaklık hassasiyeti dikkate alındığında bu görev için daha güvenli
ve uygun bir tercih olarak değerlendirilmiştir.

# 🏁 Kodunuzu kontrol edin ve notebook'unuzu gönderin

In [7]:
from nbresult import ChallengeResult

result = ChallengeResult(
    'loss_functions',
    r2 = r2,
    r2_mae = r2_mae,
    max_error = max_error_celsius,
    max_error_mae = max_error_mae
)

result.write()
print(result.check())


============================= test session starts ==============================
platform linux -- Python 3.12.9, pytest-8.3.4, pluggy-1.5.0 -- /home/selen/.pyenv/versions/3.12.9/envs/workintech/bin/python
cachedir: .pytest_cache
rootdir: /home/selen/data/S16D4-S-loss-functions/tests
plugins: anyio-4.8.0, typeguard-4.4.2, dash-3.3.0
collecting ... collected 3 items

test_loss_functions.py::TestLossFunctions::test_max_error_order PASSED   [ 33%]
test_loss_functions.py::TestLossFunctions::test_r2 PASSED                [ 66%]
test_loss_functions.py::TestLossFunctions::test_r2_mae PASSED            [100%]

============================== 3 passed in 0.14s ===============================


💯 You can commit your code:

git add tests/loss_functions.pickle

git commit -m 'Completed loss_functions step'

git push origin master

